In [1]:
%load_ext autoreload
%autoreload 2
from tf_activation import DeepGraph
from tf_activation.models import mnist_cff as mnist_model

import networkx as nx
import numpy as np
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
mnist_map = {
    0: 3,
    1: 2,
    2: 1,
    3: 18,
    4: 4,
    5: 8,
    6: 11,
    7: 0,
    8: 61,
    9: 7
}


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
x = tf.placeholder(tf.float32, [None, 784])

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

# Build the graph for the deep net
net, keep_prob = mnist_model.deepnn(x)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=net['y_conv']))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(net['y_conv'], 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [4]:
dg = DeepGraph()

In [ ]:
saver = tf.train.Saver()
im = mnist.test.images[mnist_map[7]]
lab = mnist.test.labels[mnist_map[7]]

with tf.Session() as sess:
    saver.restore(sess, '../logdir/models/mnist_cff_2000.ckpt')
    
    this_im = np.reshape(im, (1, im.shape[0]))
    this_lab = np.reshape(lab, (1, lab.shape[0]))
    
    print(accuracy.eval(feed_dict={x:this_im, y_:this_lab, keep_prob: 1.0}))
    
    conv1 = {}
    conv1['W'] = sess.run(net['W_conv1'])
    conv1['i'] = sess.run(net['input'], feed_dict={x:this_im})
    conv1['o'] = sess.run(net['h_conv1'], feed_dict={x:this_im})
        
    fc1 = {}
    fc1['W'] = sess.run(net['W_fc1'])
    fc1['i'] = conv1['o']
    fc1['o'] = sess.run(net['h_fc1'], feed_dict={x:this_im})
        
    fc2 = {}
    fc2['W'] = sess.run(net['W_fc2'])
    fc2['i'] = sess.run(net['h_fc1_drop'], feed_dict={x:this_im, keep_prob: 1.0})
    fc2['o'] = sess.run(net['y_conv'], feed_dict={x:this_im,  keep_prob: 1.0})
        
    dg.add_conv_layer(conv1['i'], conv1['W'], conv1['o'], 0, 1, [1,1,1,1])
    dg.add_fc_layer(fc1['i'], fc1['W'], fc1['o'], 1, 2)
    dg.add_fc_layer(fc2['i'], fc2['W'], fc2['o'], 2, 3)
    
    del conv1
    del fc1
    del fc2
    

INFO:tensorflow:Restoring parameters from ../logdir/models/mnist_cff_2000.ckpt
1.0
((1, 28, 28, 32), (25088, 1024), (1, 1024))
((1, 1024), (1024, 10), (1, 10))


In [ ]:
dg.connect_layers()